# Development with Langchain

## <img src="https://python.langchain.com/v0.1/img/brand/wordmark.png" alt="Langchain" width="200"/> Langchain

### Langchain is open source orchestration framework for the development of applications that use large language models


1. LLM : 
    Large Language Models (LLMs) are a core component of LangChain. LangChain does not serve its own LLMs, but rather provides a standard interface for interacting with many different LLMs

### Getting started with connecting to an LLM model

In [ ]:
%pip install langchain langchain-core langchain-community langchain-openai langchain-chroma langchainhub tavily-python sqlalchemy==1.4.46 snowflake-sqlalchemy snowflake-connector-python

#### OpenAI

In [ ]:
from langchain_openai import OpenAI
# pass in your openai api key as environment variable OPENAI_API_KEY
openai_model = OpenAI()

In [ ]:
result = openai_model.invoke("what are the SOLID principles in programming?")
print(result)

#### Locla LLM
For local llm download [Ollama](https://ollama.com/) and after installation run 
`ollama run mistral` which will expose the model on http://localhost:11434


In [ ]:
from langchain_community.llms import Ollama

mistral_model = Ollama(model="mistral")
llama3_model = Ollama(model="llama3")

In [ ]:
result = mistral_model.invoke("what are the SOLID principles in programming?")
print(result)

In [ ]:
result = llama3_model.invoke("what are the SOLID principles in programming?")
print(result)

### Prompts
Prompts are set of instructions given to a large language model. The prompt template class in langchain formalizes the composition of prompt without the need to manually hardcode context and query

In [ ]:
from langchain_core.prompts.prompt import PromptTemplate

prompt = PromptTemplate(
    template="Generate me a list of {count} catchy and unique domain names for a {business}",
    input_variables=["count", "business"]
)

result = openai_model.invoke(prompt.invoke({"count": 7, "business": "Shawarma Joint"}))
print(result)

Prompts with Structured output 

In [ ]:
from langchain.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field, validator

# Define your desired data structure.
class DomainNameList(BaseModel):
    domain_names: list[str] = Field(description="A list of domain names .")

parser = PydanticOutputParser(pydantic_object=DomainNameList)

prompt = PromptTemplate(
    template="Generate me a list of {count} domain names for a {business} . output format: {format_instructions}",
    input_variables=["count", "business"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | openai_model | parser
output = chain.invoke({"count": 6,"business": "Chipotle joint"})
print(output)
print(type(output))

### Embeddings
Every llm support embeddings

In [ ]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_openai import OpenAIEmbeddings

openai_embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
mistral_embeddings = OllamaEmbeddings(model="mistral")

In [ ]:
result = openai_embeddings.embed_query("what are the SOLID principles in programming?")
print(result)

### Indexes / Vector databases

In [ ]:
# Already ran the below code to generate the embeddings
import os
from langchain_community.document_loaders import ConfluenceLoader

loader = ConfluenceLoader(
    url="https://accountname.atlassian.net/wiki", username="username", api_key=os.environ["ATLASSIAN_API_KEY"],
    space_key="confluence_space",
    include_attachments=False,
    limit=5
)
# documents = loader.load() 

In [ ]:
from langchain_chroma import Chroma
db = Chroma(persist_directory="./chroma_db", embedding_function=mistral_embeddings,collection_name="confluence_docs")

In [ ]:
db.similarity_search("What is Resources are available?")

### Retrieval Augmented Generation (Rag) 
RAG is a technique for augmenting LLM knowledge with additional data. 
LLMs can reason about wide-ranging topics, but their knowledge is limited to the public data up to a specific point in time that they were trained on.
If you want to build AI applications that can reason about private data or data introduced after a model's cutoff date, you need to augment the knowledge of the model with the specific information it needs. 
The process of bringing the appropriate information and inserting it into the model prompt is known as Retrieval Augmented Generation (RAG)

#### Performing RAG on confluence documents

In [ ]:
from langchain import hub

retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")
for message in retrieval_qa_chat_prompt.messages:
    print(message)


In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

retriever = db.as_retriever()
combine_docs_chain = create_stuff_documents_chain(
    mistral_model, retrieval_qa_chat_prompt
)
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)

In [ ]:
result = retrieval_chain.invoke({"input": "What tools and resources are available in digital partnerships space? Give a list of key tools and resources."})
# what is vista x wix plus

In [ ]:
for message in result:
    if isinstance(result[message], list):
        print(message + ": " + ', '.join(map(str, result[message])))
    else:
        print(message + ": " + str(result[message]))

### Agents and tools
The core idea of agents is to use a language model to choose a sequence of actions to take. In chains, a sequence of actions is hardcoded (in code). In agents, a language model is used as a reasoning engine to determine which actions to take and in which order.

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_openai import ChatOpenAI # Need a model with chat capabilities


llm = ChatOpenAI(temperature=0) # Intialize the chat LLM
# Define the instructions for the agent
instructions = """You are an assistant."""
base_prompt = hub.pull("langchain-ai/openai-functions-template")
prompt = base_prompt.partial(instructions=instructions)

# Define the tools
tool = TavilySearchResults() # Search engine tool
tavily_tool = TavilySearchResults()
tools = [tavily_tool]

# Create the agent
agent = create_openai_functions_agent(llm, tools, prompt)

# Create the agent executor
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
)


In [ ]:
#agent_executor.invoke({"input": "Whats latest in react js 19?"})
agent_executor.invoke({"input": "What is the weather in Burlington Ontario?"})